In [ ]:
import numpy as np
import json

In [ ]:
# Layer Class
class Layer:
    def __init__(self, num_neuron: int, activation: str, weights: np.array, bias: np.array):
        self.num_neuron = num_neuron
        self.weights = weights
        self.bias = bias
        self.activation = activation
        if activation == 'linear':
            self.function = lambda x: x
            self.derivative = lambda x: 1
        elif activation == 'relu':
            self.function = lambda x: np.maximum(0, x)
            self.derivative = lambda x: np.where(x > 0, 1, 0)
        elif activation == 'sigmoid':
            self.function = lambda x: 1 / (1 + np.exp(-x))
            self.derivative = lambda x: self.function(x) * (1 - self.function(x))
        elif activation == 'softmax':
            self.function = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
            self.derivative = lambda x: self.function(x) - self.function(x) ** 2
        else:
            raise ValueError('Invalid activation function')

    def forward(self, input: np.array):
        self.input = input
        self.net = np.dot(input, self.weights) + self.bias
        self.output = self.function(self.net)
        return self.output

In [ ]:
class FFNN:
    def __init__(self, input_size: int, layers: list, learning_rate: float, epochs: int, batch_size: int, threshold: float, initial_weights: list):
        self.input_size = input_size
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.threshold = threshold
        self.loss = []
        self.initial_weights = initial_weights
        self.final_weights = []
        self.stopped_by = ""

    def add_layer(self, layer: Layer):
        self.layers.append(layer)

    def forward(self, input: np.array):
        output = input
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, target: np.array):
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            if i == len(self.layers) - 1:
                if layer.activation == 'softmax':
                    layer.delta = layer.output - target
                else:
                    error = layer.output - target
                    layer.delta = error * layer.derivative(layer.net)
            else:
                layer.delta = np.dot(self.layers[i+1].delta, self.layers[i+1].weights.T) * layer.derivative(layer.net)
            layer.weights -= self.learning_rate * np.dot(layer.input.T, layer.delta)
            layer.bias -= self.learning_rate * np.squeeze(np.sum(layer.delta, axis=0))

    def train(self, X_train: np.array, y_train: np.array):
        for i in range(len(self.layers)):
            self.layers[i].weights = np.array(self.initial_weights[i][1:])
            self.layers[i].bias = np.array(self.initial_weights[i][0])

        for epoch in range(self.epochs):
            error_epoch = 0
            # Mini-batch
            for i in range(0, X_train.shape[0], self.batch_size):
                X_batch = X_train[i:i + self.batch_size]
                y_batch = y_train[i:i + self.batch_size]
                output = self.forward(X_batch)
                self.backward(y_batch)
                if self.layers[-1].activation == 'softmax':
                    error_batch = np.mean(-y_batch * np.log(output))
                else:
                    error_batch = np.mean((output - y_batch)**2)
                error_epoch += error_batch
            error_epoch /= (X_train.shape[0] / self.batch_size)
            self.loss.append(error_epoch)
            print(f"Epoch {epoch+1}/{self.epochs}, Error: {error_epoch:.6f}")
            if error_epoch <= self.threshold:
                print(f"Training stopped at epoch {epoch+1} with error {error_epoch:.6f}")
                self.stopped_by = "error_threshold"
                break
        if epoch == self.epochs - 1:
            self.stopped_by = "max_iteration"
        self.final_weights = [[layer.bias.tolist()] + layer.weights.tolist() for layer in self.layers]

In [ ]:
input_file = str(input("Enter the input file name (JSON only): "))
print(f"Test case: {input_file}.json\n")
with open(f"test-case/{input_file}.json", "r") as file:
    model = json.load(file)

input_size = model["case"]["model"]["input_size"]
layers = []
for layer in model["case"]["model"]["layers"]:
    layers.append(Layer(layer["number_of_neurons"], layer["activation_function"], None, None))
learning_rate = model["case"]["learning_parameters"]["learning_rate"]
epochs = model["case"]["learning_parameters"]["max_iteration"]
batch_size = model["case"]["learning_parameters"]["batch_size"]
threshold = model["case"]["learning_parameters"]["error_threshold"]
initial_weights = model["case"]["initial_weights"]
ffnn = FFNN(input_size, layers, learning_rate, epochs, batch_size, threshold, initial_weights)

X_train = np.array(model["case"]["input"])
y_train = np.array(model["case"]["target"])
ffnn.train(X_train, y_train)

In [ ]:
expected_stopped_by = model["expect"]["stopped_by"]
if ffnn.stopped_by != expected_stopped_by:
    print(f"Test case failed: Stopped by {ffnn.stopped_by}, expected {expected_stopped_by}")
else:
    print(f"Stopped by {ffnn.stopped_by}")

print("Final Weights:")
for i, weights in enumerate(ffnn.final_weights):
    for j, row in enumerate(weights):
        print("[" + " ".join([f"{val:.6f}" for val in row]) + "]")
    print()

if "final_weights" in model["expect"]:
    expected_weights = model["expect"]["final_weights"]
    for i in range(len(expected_weights)):
        diff = np.array(ffnn.final_weights[i]) - np.array(expected_weights[i])
        sse = np.sum(diff**2)
        if sse > 1e-7:
            print(f"Test case failed: Final weight does not match. SSE = {sse:.8f}")
            break
    else:
        print("All test case passed!")
else:
    print("No expected final weights provided in the test case.")